# Pre-processing

In [ ]:
####################################
##        ACTION REQUIRED         ##
####################################

# Update the following line with the starting and ending nodes of your choice:

start_node, end_node = 50, 823

####################################
##    END OF ACTION REQUIRED      ##
####################################

# Configurating Hadoop

sc._jsc.hadoopConfiguration().set('mapreduce.fileoutputcommitter.marksuccessfuljobs', 'false') # Preventing "SUCCESS" file to be created when writting a text file


# Defining custom functions

def formatInput(raw_file):
    ## Creating key-value pairs: key = node A ; value = (node B, distance from A to B)
    file = raw_file.map(lambda x: (x.split('\t')[0], (x.split('\t')[2], x.split('\t')[1])))
    
    ## Grouping by key
    file = file.groupByKey()
    
    ## Adding the distance (1) and the path (2)
    file = file.map(lambda x: (x[0], 
                               0 if x[0] == str(start_node) else float('inf'), ##(1)
                               list(x[1]),
                               " ")) ## (2)
    return file


def checkExistence(file, start_node, end_node):
    start_node_exists = not file.filter(lambda x: str(start_node) == x[0]).isEmpty()
    end_node_exists = not file.map(lambda xs: sum([1 if  x == str(end_node) else 0 for x in xs])).filter(lambda x: x!=0).isEmpty()
    return start_node_exists, end_node_exists



# Transforming our file

file = sc.textFile("../data/graph.txt")
file = formatInput(file)
input_nodes_exist = checkExistence(file, start_node, end_node)

if input_nodes_exist[0] & input_nodes_exist[1]: # both starting and ending nodes exist
    try:
        file.coalesce(1).saveAsTextFile("input") # saving the new text file
    except:
        print('Error: "Input" directory already exists. You need to remove it.')
elif input_nodes_exist[0]:
    print("Your ending node does not exist")
else:
    print("Your starting node does not exist")